In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Data Preparation

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
x = [[10, 5],
     [9, 5],
     [3, 3],
     [2, 4],
     [4, 1]]
y = [0, 0, 1, 1, 2]
x = torch.FloatTensor(x)
y = torch.LongTensor(y)

In [ ]:
x.shape

torch.Size([5, 2])

In [ ]:
y.shape

torch.Size([5])

In [ ]:
W = torch.zeros((2, 3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)

In [ ]:
hypothesis = F.softmax(x.matmul(W) + b, dim=1) #Same as logistic regression

In [ ]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [ ]:
optimizer = optim.SGD([W, b], lr=0.1)

In [ ]:
for epoch in range(1001):
  # hypothesis = F.softmax(x, dim=1)
  # y_one_hot = torch.zeros_like(hypothesis)
  # y_one_hot.scatter_(1, y.unsqueeze(1), 1)  #The first 1 means dimention=1, y.unsqueeze(1) means change y(5) to changey(5,1)
  # cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

  hypothesis = F.softmax(x.matmul(W) + b, dim=1) #Same as logistic regression
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1, y.unsqueeze(1), 1) #The first 1 means dimention=1, y.unsqueeze(1) means change y(5) to changey(5,1)
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.2f}'.format(epoch, 1000, cost.item()))

Epoch    0/1000 Cost: 1.10
Epoch  100/1000 Cost: 0.73
Epoch  200/1000 Cost: 0.66
Epoch  300/1000 Cost: 0.62
Epoch  400/1000 Cost: 0.60
Epoch  500/1000 Cost: 0.59
Epoch  600/1000 Cost: 0.58
Epoch  700/1000 Cost: 0.58
Epoch  800/1000 Cost: 0.57
Epoch  900/1000 Cost: 0.57
Epoch 1000/1000 Cost: 0.57


# PyTorch Coding-V2

In [ ]:
#High-level implementation with nn.Module

class SoftmaxClassfierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2, 3)

  def forward(self, x):
    return self.linear(x)

In [ ]:
model = SoftmaxClassfierModel()

In [ ]:
x = [[10, 5],
     [9, 5],
     [3, 3],
     [2, 4],
     [4, 1]]
y = [0, 0, 1, 1, 2]
x = torch.FloatTensor(x)
y = torch.LongTensor(y)

optimizer = optim.SGD(model.parameters(), lr=0.1) #model.parameters() means linear layer

for epoch in range (1001):
    hypothesis = model(x)

    cost = F.cross_entropy(hypothesis, y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, 1000, cost.item()))

Epoch    0/1000 Cost: 0.958955
Epoch  100/1000 Cost: 0.161244
Epoch  200/1000 Cost: 0.084836
Epoch  300/1000 Cost: 0.056866
Epoch  400/1000 Cost: 0.042631
Epoch  500/1000 Cost: 0.034057
Epoch  600/1000 Cost: 0.028341
Epoch  700/1000 Cost: 0.024263
Epoch  800/1000 Cost: 0.021208
Epoch  900/1000 Cost: 0.018836
Epoch 1000/1000 Cost: 0.016941


In [ ]:
print('#1')
print(list(model.parameters()))

#1
[Parameter containing:
tensor([[ 0.7517,  0.0937],
        [-1.4256,  2.4287],
        [ 1.4030, -2.9130]], requires_grad=True), Parameter containing:
tensor([-2.0419,  1.1628,  1.7176], requires_grad=True)]


In [ ]:
test_data = torch.FloatTensor([[5, 5]])
predict_result = model(test_data)
print("Prediction: ", F.softmax(predict_result, dim=1))

Prediction:  tensor([[1.8113e-02, 9.8188e-01, 5.9690e-06]], grad_fn=<SoftmaxBackward0>)


# TensorFlow Coding

In [ ]:
x = [[10, 5],
     [9, 5],
     [3, 3],
     [2, 4],
     [4, 1]]
y = [[1, 0, 0],
     [1, 0, 0],
     [0, 1, 0],
     [0, 1, 0],
     [0, 0, 1]]

X = tf.placeholder("float", [None, 2])
Y = tf.placeholder("float", [None, 3])

W = tf.Variable(tf.random_normal([2, 3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(1001):
  _, cost_val = sess.run([optimizer, cost], feed_dict={X: x, Y: y})

  if epoch % 200 == 0:
    print("Epoch", epoch, ": ", "Cost: ", cost_val)

print('====Testing====')
test_data = sess.run(hypothesis, feed_dict={X: [[5, 5]]})
print(test_data, sess.run(tf.argmax(test_data, 1)))

Epoch 0 :  Cost:  2.1920295
Epoch 200 :  Cost:  0.09163688
Epoch 400 :  Cost:  0.042278875
Epoch 600 :  Cost:  0.027452398
Epoch 800 :  Cost:  0.02036261
Epoch 1000 :  Cost:  0.01620557
====Testing====
[[3.9476812e-02 9.6051764e-01 5.4977195e-06]] [1]
